In [237]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA, KernelPCA
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import cohen_kappa_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import pickle

import numpy as np
import pandas as pd

In [238]:
df = pd.read_csv('data/EEG.machinelearing_data_BRMH.csv')
df.head()

,no.,sex,age,eeg.date,education,IQ,main.disorder,specific.disorder,AB.A.delta.a.FP1,AB.A.delta.b.FP2,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
0,1,M,57.0,2012.8.30,NaN,NaN,Addictive disorder,Alcohol use disorder,35.998557,21.717375,...,55.989192,16.739679,23.452271,45.678820,30.167520,16.918761,48.850427,9.422630,34.507082,28.613029
1,2,M,37.0,2012.9.6,6.0,120.0,Addictive disorder,Alcohol use disorder,13.425118,11.002916,...,45.595619,17.510824,26.777368,28.201062,57.108861,32.375401,60.351749,13.900981,57.831848,43.463261
2,3,M,32.0,2012.9.10,16.0,113.0,Addictive disorder,Alcohol use disorder,29.941780,27.544684,...,99.475453,70.654171,39.131547,69.920996,71.063644,38.534505,69.908764,27.180532,64.803155,31.485799
3,4,M,35.0,2012.10.8,18.0,126.0,Addictive disorder,Alcohol use disorder,21.496226,21.846832,...,59.986561,63.822201,36.478254,47.117006,84.658376,24.724096,50.299349,35.319695,79.822944,41.141873
4,5,M,36.0,2012.10.18,16.0,112.0,Addictive disorder,Alcohol use disorder,37.775667,33.607679,...,61.462720,59.166097,51.465531,58.635415,80.685608,62.138436,75.888749,61.003944,87.455509,70.531662


In [250]:
df.isnull().any()

no.                      False
sex                      False
age                      False
eeg.date                 False
education                 True
                         ...  
COH.F.gamma.p.P4.r.O1    False
COH.F.gamma.p.P4.s.O2    False
COH.F.gamma.q.T6.r.O1    False
COH.F.gamma.q.T6.s.O2    False
COH.F.gamma.r.O1.s.O2    False
Length: 1149, dtype: bool

In [251]:
df = df.fillna(method='ffill')

In [252]:
df.columns

Index(['no.', 'sex', 'age', 'eeg.date', 'education', 'IQ', 'main.disorder',
       'specific.disorder', 'AB.A.delta.a.FP1', 'AB.A.delta.b.FP2',
       ...
       'COH.F.gamma.o.Pz.p.P4', 'COH.F.gamma.o.Pz.q.T6',
       'COH.F.gamma.o.Pz.r.O1', 'COH.F.gamma.o.Pz.s.O2',
       'COH.F.gamma.p.P4.q.T6', 'COH.F.gamma.p.P4.r.O1',
       'COH.F.gamma.p.P4.s.O2', 'COH.F.gamma.q.T6.r.O1',
       'COH.F.gamma.q.T6.s.O2', 'COH.F.gamma.r.O1.s.O2'],
      dtype='object', length=1149)

In [253]:
independent_variables = pd.DataFrame(df.drop(columns = ['no.', 'sex', 'age', 'eeg.date', 'education', 'IQ', 'main.disorder',
       'specific.disorder'],inplace = False))
independent_variables.describe()

,AB.A.delta.a.FP1,AB.A.delta.b.FP2,AB.A.delta.c.F7,AB.A.delta.d.F3,AB.A.delta.e.Fz,AB.A.delta.f.F4,AB.A.delta.g.F8,AB.A.delta.h.T3,AB.A.delta.i.C3,AB.A.delta.j.Cz,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
count,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,...,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000
mean,20.182936,21.177584,17.749553,18.901698,20.447079,19.490329,17.042218,11.790176,16.405732,19.566879,...,75.889633,55.049113,56.959796,60.739169,69.829254,47.862489,66.832798,39.301406,66.153213,57.056207
std,11.282022,12.230662,10.003598,9.079482,9.742912,9.252500,9.272096,7.347929,8.246200,9.525142,...,15.861641,19.541956,18.326785,18.052887,17.725321,19.685722,17.028701,20.790933,18.088548,19.600107
min,3.272260,3.244199,3.050507,3.412618,5.066593,4.048931,2.552213,2.128977,2.213470,3.149313,...,12.611954,0.519048,6.926792,0.708008,2.421748,0.036664,1.032207,1.228502,0.363268,3.988805
25%,12.784872,13.019269,11.134327,12.460586,13.548645,12.637717,10.381770,7.042545,10.686639,13.130418,...,68.014443,41.607506,43.682444,48.374883,58.937785,32.581046,55.872070,22.049743,54.710605,43.955229
50%,17.065286,17.838251,15.541469,16.733004,18.065276,17.432568,15.050377,10.044354,14.525494,17.245633,...,78.647712,55.158858,56.657348,61.257972,72.298636,45.719426,68.238375,36.549938,67.988937,57.515871
75%,24.492760,25.654394,21.623961,23.347900,25.573096,23.956748,20.810308,14.246874,20.271380,23.792159,...,87.398387,69.740640,70.649245,73.979100,83.066877,62.081839,79.192418,54.169209,79.527764,71.626382
max,92.826192,101.515687,91.373456,82.544167,101.566662,77.197502,69.071230,77.283412,65.932521,88.474026,...,99.678649,100.000000,99.307895,100.000000,99.581629,98.720067,99.650154,98.413320,100.000000,99.287092


In [254]:
from sklearn.impute import SimpleImputer
impute_mean = SimpleImputer(missing_values=np.nan, strategy ='mean')
impute_mean.fit_transform(independent_variables)

array([[35.998557, 21.717375, 21.51828 , ...,  9.42263 , 34.507082,
        28.613029],
       [13.425118, 11.002916, 11.942516, ..., 13.900981, 57.831848,
        43.463261],
       [29.94178 , 27.544684, 17.150159, ..., 27.180532, 64.803155,
        31.485799],
       ...,
       [28.781317, 32.36923 , 11.717778, ...,  7.045821, 41.962451,
        19.092111],
       [19.9291  , 25.196375, 14.445391, ..., 27.1765  , 27.529522,
        20.028446],
       [65.195346, 69.241972, 35.361363, ..., 47.810386, 80.166825,
        64.380273]])

In [255]:
np.isfinite(independent_variables).all()

AB.A.delta.a.FP1         True
AB.A.delta.b.FP2         True
AB.A.delta.c.F7          True
AB.A.delta.d.F3          True
AB.A.delta.e.Fz          True
                         ... 
COH.F.gamma.p.P4.r.O1    True
COH.F.gamma.p.P4.s.O2    True
COH.F.gamma.q.T6.r.O1    True
COH.F.gamma.q.T6.s.O2    True
COH.F.gamma.r.O1.s.O2    True
Length: 1141, dtype: bool

In [256]:
target = pd.DataFrame(df['specific.disorder'])
target_names = df['specific.disorder'].unique() 
target_names

array(['Alcohol use disorder', 'Acute stress disorder',
       'Depressive disorder', 'Healthy control',
       'Behavioral addiction disorder', 'Obsessive compulsitve disorder',
       'Schizophrenia', 'Panic disorder', 'Social anxiety disorder',
       'Posttraumatic stress disorder', 'Adjustment disorder',
       'Bipolar disorder'], dtype=object)

In [257]:
#from sklearn.impute import SimpleImputer
#impute_mean = SimpleImputer(missing_values=np.nan, strategy ='mean')
#invarfit = impute_mean.fit(independent_variables)
#invartrans = impute_mean.transform(invarfit)

In [258]:
independent_variables.fillna(method = 'ffill', inplace = True)
target.fillna(method = 'ffill', inplace = True)

In [259]:
X_train, X_test, y_train, y_test = train_test_split(independent_variables, target, test_size=0.2, random_state=42)

In [260]:
X_test

,AB.A.delta.a.FP1,AB.A.delta.b.FP2,AB.A.delta.c.F7,AB.A.delta.d.F3,AB.A.delta.e.Fz,AB.A.delta.f.F4,AB.A.delta.g.F8,AB.A.delta.h.T3,AB.A.delta.i.C3,AB.A.delta.j.Cz,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
848,34.820107,35.085551,27.252116,29.056442,38.634672,36.366347,28.661629,15.240781,40.225591,38.198875,...,84.826939,64.134452,75.336179,76.576526,74.856292,68.741795,79.043187,62.032961,79.282698,80.363086
333,16.225331,18.846511,15.822599,21.702640,24.386782,20.753664,14.713953,13.451991,16.339981,17.991201,...,90.227855,74.855314,80.771301,79.584291,89.402741,73.302929,88.206628,65.647020,87.795637,77.529607
280,11.971083,11.374465,8.205410,9.041619,13.420321,11.523788,6.641631,4.607822,10.945109,15.001368,...,91.150863,78.359575,64.861521,74.389105,89.863923,50.322649,75.469583,49.440177,84.022887,69.921675
30,7.444542,8.637013,9.627330,8.093987,8.225006,8.053835,9.335740,4.643903,6.719088,7.905416,...,38.160508,21.319727,32.967416,33.401386,37.406879,27.080760,39.050046,14.826695,32.285940,41.015604
168,10.957138,15.031378,8.916309,11.316517,13.178424,11.948546,12.852749,5.294943,11.079374,13.607071,...,84.592867,56.558457,36.026846,54.528173,80.733374,31.961799,71.590967,21.863386,69.864630,38.386405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,17.845402,19.261821,18.129482,16.475350,17.222169,18.367626,28.229975,12.294245,18.239670,16.359816,...,96.810006,88.851606,95.967370,96.496343,93.808248,93.450709,97.542257,86.033005,94.431469,95.387236
541,31.857373,40.180357,31.656775,41.717763,42.085019,34.322759,30.297688,20.413057,47.008136,32.778350,...,72.333472,34.552786,36.000105,46.653722,64.402299,25.829763,57.835864,16.411582,56.148682,54.521541
850,23.488215,21.986901,13.094994,18.425377,19.959232,17.903340,19.074081,6.383472,14.525494,16.669198,...,62.445722,27.349752,42.011781,43.830968,60.370310,23.074387,51.450177,10.975746,46.648027,36.795983
109,8.189847,7.782097,7.928687,10.289899,12.891478,9.891710,7.281608,5.886675,13.692340,14.969934,...,78.107126,56.727190,59.110962,66.427124,73.163979,50.109266,73.605611,38.036346,72.609308,59.683616


In [261]:
print("Train features shape", X_train.shape)
print("Train target shape", y_train.shape)
print("Test features shape", X_test.shape)
print("Test target shape", y_test.shape)

Train features shape (756, 1141)
Train target shape (756, 1)
Test features shape (189, 1141)
Test target shape (189, 1)


In [262]:
from sklearn.preprocessing import Binarizer
scaler = Binarizer()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_test

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [202]:
impute_mean.fit_transform(X_test)
impute_mean.fit_transform(X_train)

array([[0.26842123, 0.21660615, 0.18197512, ..., 0.22096079, 0.35013258,
        0.70707986],
       [0.23408036, 0.22739875, 0.21343299, ..., 0.44838581, 0.62058889,
        0.67931567],
       [0.11977932, 0.11970867, 0.0725526 , ..., 0.31751835, 0.74038603,
        0.51605579],
       ...,
       [0.18199787, 0.15005221, 0.1702611 , ..., 0.25375411, 0.68558689,
        0.2643316 ],
       [0.14524424, 0.1404928 , 0.19935073, ..., 0.32928682, 0.64695148,
        0.50572519],
       [0.18353069, 0.13819067, 0.16462479, ..., 0.46056575, 0.69353209,
        0.70199707]])

In [204]:
np.sum(X_train)

nan

In [178]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').